In [1]:
import torch
from torchvision import transforms #this contain composed transfrom
from torch.utils.data import Dataset
import numpy as np

In [146]:
#Step0: Build a dataset that allow transfrom attubite

class WineDataset(Dataset):
    #here we add an attribute of transform, default state is None
    def __init__(self, transform = None):
        wine_numpy = np.loadtxt('wine.csv', delimiter = ",", dtype = np.float32, skiprows = 1)
        self.samples = wine_numpy.shape[0]
        #extract the features and classes of dataset
        self.wine_features = wine_numpy[:, 1:]
        #in this case it must be [:, [0]] or it will not be a numpy array
        self.wine_labels = wine_numpy[:, [0]]
        #save transfrom as self variable, that can be used when getting items
        self.transform = transform

    def __getitem__(self, index):
        item = self.wine_features[index], self.wine_labels[index]
        #if transform is not None
        if self.transform:
            item = self.transform(item)
        return item   

    def __len__(self):
        return self.samples

In [147]:
#Step1:Customize our transfrom

class TensorTransform:
    def __call__(self, input):
        #unpack the input
        features, labels = input
        #print(type(features))
        return torch.from_numpy(features), torch.from_numpy(labels)


class MulTransform:
    #if the transform requires a value, you must specify in __init__()
    def __init__(self, mul):
        self.mul = mul

    def __call__(self, input):
        features, labels = input
        features = features * self.mul
        return features, labels

#make a composed transfrom with two custimize transform       
#remember to give mul attribute to MulTransform
composed_transform = transforms.Compose([TensorTransform(), MulTransform(10)])


In [148]:
#Step2:Vertify datatype

dataset = WineDataset(transform = TensorTransform())
feature, label = dataset[0]
print(feature)
print(type(feature))

dataset = WineDataset(transform = composed_transform)
feature, label = dataset[0]
print(feature)
print(type(feature))

#Note: Two np arrays concatenate will become a tuple, in which any type of arrays can be concatenate
#When packing and unpacking array pairs, make sure to operate a tensor of numpy rather than a tuple
#Note: When extracting a line in np.array make sure to use [i] as indes rather than i
#if you use i as a index, this return a list, if you use [i] as a index, this return an array

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
<class 'torch.Tensor'>
tensor([1.4230e+02, 1.7100e+01, 2.4300e+01, 1.5600e+02, 1.2700e+03, 2.8000e+01,
        3.0600e+01, 2.8000e+00, 2.2900e+01, 5.6400e+01, 1.0400e+01, 3.9200e+01,
        1.0650e+04])
<class 'torch.Tensor'>
